В этом ноутбуке обучим модель ALBERT распознавать спам в SMS

In [1]:
model_name = 'albert'
train_dataset_name = 'spam sms'

In [2]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 45.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [4]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_sms.csv', encoding = "ISO-8859-1")
df

Mounted at /content/drive


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [5]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df.columns = ['IS_SPAM', 'DATA_COLUMN']
df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)

In [6]:
df

,IS_SPAM,DATA_COLUMN
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [7]:
df['IS_SPAM'].value_counts()

0    4825
1     747
Name: IS_SPAM, dtype: int64

In [8]:
df_positive = df[df['IS_SPAM']==1]

In [9]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [10]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [11]:
df_negative_test.shape

(2412, 2)

In [12]:
df_positive_test.shape

(373, 2)

In [13]:
df_positive_test

,IS_SPAM,DATA_COLUMN
2728,1,Urgent Please call 09066612661 from landline. ...
2729,1,Urgent! Please call 09066612661 from your land...
2741,1,I don't know u and u don't know me. Send CHAT ...
2766,1,Married local women looking for discreet actio...
2769,1,Burger King - Wanna play footy at a top stadiu...
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


In [14]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [15]:
df_balanced_test.sample(10)

,IS_SPAM,DATA_COLUMN
4163,0,"Its ok, called mom instead have fun"
3016,0,One of the joys in lifeis waking up each daywi...
3602,0,"I'm not sure, I was just checking out what was..."
2928,0,Where wuld I be without my baby? The thought a...
3741,0,Hey i'm bored... So i'm thinking of u... So wa...
4806,1,PRIVATE! Your 2004 Account Statement for 07849...
4546,0,Hey mate. Spoke to the mag people. WeÛ÷re on....
3954,0,I knew it... U slept v late yest? Wake up so l...
3501,0,I will come to ur home now
3705,0,Reading gud habit.. Nan bari hudgi yorge patai...


In [16]:
df_balanced_test['IS_SPAM'].value_counts()

0    2412
1     373
Name: IS_SPAM, dtype: int64

In [17]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [18]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [19]:
df_balanced_train['IS_SPAM'].value_counts()

0    2412
1     373
Name: IS_SPAM, dtype: int64

In [20]:
df_balanced_train.sample(10)

,IS_SPAM,DATA_COLUMN
2674,0,I am 6 ft. We will be a good combination!
2450,0,K..give back my thanks.
1549,0,Dude sux for snake. He got old and raiden got ...
2730,0,I havent lei.. Next mon can?
21,0,IÛ÷m going to try for 2 months ha ha only joking
143,0,I know you are. Can you pls open the back?
2245,0,How did you find out in a way that didn't incl...
2041,0,ÌÏ dun wan to watch infernal affair?
1322,0,Oh just getting even with u.... u?
147,0,Ummma.will call after check in.our life will b...


In [21]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['IS_SPAM'].squeeze()

In [22]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['IS_SPAM'].squeeze()

Работаем с моделью

In [23]:
albert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")


In [24]:
albert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3")

In [25]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = albert_preprocess(text_input)
outputs = albert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [26]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [27]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [29]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [30]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
88/88 [==============================] - 38s 324ms/step - loss: 0.3592 - accuracy: 0.8596 - precision: 0.3393 - recall: 0.0509
Epoch 2/15
88/88 [==============================] - 30s 341ms/step - loss: 0.2212 - accuracy: 0.9138 - precision: 0.8446 - recall: 0.4370
Epoch 3/15
88/88 [==============================] - 30s 339ms/step - loss: 0.1713 - accuracy: 0.9354 - precision: 0.9004 - recall: 0.5818
Epoch 4/15
88/88 [==============================] - 29s 334ms/step - loss: 0.1503 - accuracy: 0.9440 - precision: 0.8974 - recall: 0.6568
Epoch 5/15
88/88 [==============================] - 30s 337ms/step - loss: 0.1315 - accuracy: 0.9573 - precision: 0.9123 - recall: 0.7534
Epoch 6/15
88/88 [==============================] - 30s 342ms/step - loss: 0.1194 - accuracy: 0.9616 - precision: 0.9236 - recall: 0.7775
Epoch 7/15
88/88 [==============================] - 30s 336ms/step - loss: 0.1067 - accuracy: 0.9666 - precision: 0.9268 - recall: 0.8150
Epoch 8/15
88/88 [================

In [31]:
y_predicted = model.predict(X_test)
y_predicted


88/88 [==============================] - 30s 335ms/step


array([[0.05251721],
       [0.00473549],
       [0.02193624],
       ...,
       [0.3704737 ],
       [0.7632304 ],
       [0.6776418 ]], dtype=float32)

In [32]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [33]:
y_test

2795    0
2796    0
2797    0
2798    0
2799    0
       ..
5537    1
5540    1
5547    1
5566    1
5567    1
Name: IS_SPAM, Length: 2785, dtype: int64

In [34]:
accuracy_score(y_test, y_predicted)

0.9755834829443447

In [35]:
precision_score(y_test, y_predicted)


0.9579579579579579

In [36]:
recall_score(y_test, y_predicted)

0.8552278820375335

In [37]:
f1_score(y_test, y_predicted)

0.9036827195467422

In [38]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [39]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [40]:
df_results_on_test

,accuracy,precision,recall,f1_score
albert trained on spam sms and tested on spam sms dataset,0.975583,0.957958,0.855228,0.903683


Сохраним обученную модель

In [41]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/albert_trained_on_spam_sms_19_january'

In [42]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/albert_trained_on_spam_sms_19_january


In [43]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [44]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [45]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [46]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [47]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [48]:
df_with_train_quality

,accuracy,precision,recall,f1_score
albert trained on spam sms quality on train dataset,0.97702,0.937677,0.887399,0.911846


In [49]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [50]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [51]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')